In [1]:
import pandas as pd

In [2]:
#importing videogame database
vgdf = pd.read_csv("vgsales.csv")
vgdf

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [3]:
#removing older platforms of which we can't gather reviews online per Tanner
to_drop = ['NES', 'SNES', 'SAT', '2600', 'GB', 'GEN', 'NG', 'SCD', 'WS', '3DO', 'TG16', 'GG', 'PCFX']
vgdf_plat_clean = vgdf[~vgdf['Platform'].isin(to_drop)]
vgdf_plat_clean

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
6,7,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01
7,8,Wii Play,Wii,2006.0,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


#### Publisher ID Table

In [4]:
#creating publisher table with id determined by its index #
vg_publisher_df = pd.DataFrame({'publisher':vgdf_plat_clean['Publisher'].unique()})
vg_publisher_df.index.name='publisher_id'
vg_publisher_df

,publisher
publisher_id,
0,Nintendo
1,Microsoft Game Studios
2,Take-Two Interactive
3,Sony Computer Entertainment
4,Activision
...,...
520,Inti Creates
521,Takuyo
522,Interchannel-Holon


#### Platform ID Table

In [5]:
#creating platform table with id determined by its index #
vg_platform_df = pd.DataFrame({'platform':vgdf_plat_clean['Platform'].unique()})
vg_platform_df.index.name='platform_id'
vg_platform_df

,platform
platform_id,
0,Wii
1,DS
2,X360
3,PS3
4,PS2
5,GBA
6,3DS
7,PS4
8,N64


#### Genre ID Table

In [6]:
vg_genre_df = pd.DataFrame({'genre':vgdf_plat_clean['Genre'].unique()})
vg_genre_df.index.name='genre_id'
vg_genre_df

,genre
genre_id,
0,Sports
1,Racing
2,Platform
3,Misc
4,Simulation
5,Action
6,Role-Playing
7,Puzzle
8,Shooter


### Replace platform names with id's - TEST

In [7]:
#Test
#Creating dictionary with platform_id and platform - then inversing key/value to format a dictionary that can then be used to
#replace platform with id in other cleaned tables

platformdict = vg_platform_df.to_dict('index')
platformdict

for key in platformdict:
    platformdict[key] = platformdict[key]["platform"]
    
platformdict
    
platformdict_s = {value:key for key, value in platformdict.items()}
platformdict_s    

{'Wii': 0,
 'DS': 1,
 'X360': 2,
 'PS3': 3,
 'PS2': 4,
 'GBA': 5,
 '3DS': 6,
 'PS4': 7,
 'N64': 8,
 'PS': 9,
 'XB': 10,
 'PC': 11,
 'PSP': 12,
 'XOne': 13,
 'GC': 14,
 'WiiU': 15,
 'DC': 16,
 'PSV': 17}

In [8]:
#test platform_id replace in Platform column 
vgdf_replacer = {
    "Platform": platformdict_s
}
vgdf.replace(vgdf_replacer)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,0,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,0,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,0,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,5,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,14,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,4,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,1,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


### Add id's to tables formatted by Tyler - Game Title, Sales

#### Game Title

In [39]:
#import transformed game title table in for publisher id transformation
game_title_df = pd.read_csv('title.csv')
game_title_df = game_title_df.set_index('title_id')
game_title_df

,game_title,year_released,publisher,genre
title_id,,,,
0,Wii Sports,2006.0,Nintendo,Sports
1,Mario Kart Wii,2008.0,Nintendo,Racing
2,Wii Sports Resort,2009.0,Nintendo,Sports
3,New Super Mario Bros.,2006.0,Nintendo,Platform
4,Wii Play,2006.0,Nintendo,Misc
...,...,...,...,...
10810,Chou Ezaru wa Akai Hana: Koi wa Tsuki ni Shiru...,2016.0,dramatic create,Action
10811,Eiyuu Densetsu: Sora no Kiseki Material Collec...,2007.0,Falcom Corporation,Role-Playing
10812,Plushees,2008.0,Destineer,Simulation


In [43]:
#create publisher id dict for replacing in publisher column of table
#use vg_publisher_df from before

publisherdict = vg_publisher_df.to_dict('index')
publisherdict

for key in publisherdict:
    publisherdict[key] = publisherdict[key]["publisher"]
    
publisherdict_s = {value:key for key, value in publisherdict.items()}
publisherdict_s  

{'Nintendo': 0,
 'Microsoft Game Studios': 1,
 'Take-Two Interactive': 2,
 'Sony Computer Entertainment': 3,
 'Activision': 4,
 'Ubisoft': 5,
 'Bethesda Softworks': 6,
 'Electronic Arts': 7,
 'Sega': 8,
 'SquareSoft': 9,
 '505 Games': 10,
 'GT Interactive': 11,
 'Konami Digital Entertainment': 12,
 'Sony Computer Entertainment Europe': 13,
 'Square Enix': 14,
 'LucasArts': 15,
 'Virgin Interactive': 16,
 'Warner Bros. Interactive Entertainment': 17,
 'Capcom': 18,
 'Universal Interactive': 19,
 'Eidos Interactive': 20,
 'RedOctane': 21,
 'Atari': 22,
 'Vivendi Games': 23,
 'Enix Corporation': 24,
 'Namco Bandai Games': 25,
 'Hasbro Interactive': 26,
 'THQ': 27,
 'Fox Interactive': 28,
 'Acclaim Entertainment': 29,
 'MTV Games': 30,
 'Disney Interactive Studios': 31,
 nan: 32,
 'Majesco Entertainment': 33,
 'Codemasters': 34,
 'Red Orb': 35,
 'Level 5': 36,
 'Midway Games': 37,
 'JVC': 38,
 'Deep Silver': 39,
 '989 Studios': 40,
 'NCSoft': 41,
 'UEP Systems': 42,
 'Maxis': 43,
 'Tecmo K

In [44]:
genredict = vg_genre_df.to_dict('index')

for key in genredict:
    genredict[key] = genredict[key]["genre"]
    
genredict_s = {value:key for key, value in genredict.items()}
genredict_s  

{'Sports': 0,
 'Racing': 1,
 'Platform': 2,
 'Misc': 3,
 'Simulation': 4,
 'Action': 5,
 'Role-Playing': 6,
 'Puzzle': 7,
 'Shooter': 8,
 'Fighting': 9,
 'Adventure': 10,
 'Strategy': 11}

In [63]:
#replace publisher w/ publisher_id and genre_id in game title df
pub_id_replacer = {
    "publisher": publisherdict_s,
    "genre": genredict_s
}
game_title_df = game_title_df.replace(pub_id_replacer)
game_title_df = game_title_df.rename(columns={'publisher':'publisher_id', 'genre':'genre_id'})
game_title_df

,game_title,year_released,publisher_id,genre_id
title_id,,,,
0,Wii Sports,2006.0,0,0
1,Mario Kart Wii,2008.0,0,1
2,Wii Sports Resort,2009.0,0,0
3,New Super Mario Bros.,2006.0,0,2
4,Wii Play,2006.0,0,3
...,...,...,...,...
10810,Chou Ezaru wa Akai Hana: Koi wa Tsuki ni Shiru...,2016.0,441,5
10811,Eiyuu Densetsu: Sora no Kiseki Material Collec...,2007.0,216,6
10812,Plushees,2008.0,136,4


In [64]:
game_title_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10815 entries, 0 to 10814
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   game_title     10815 non-null  object 
 1   year_released  10815 non-null  float64
 2   publisher_id   10815 non-null  int64  
 3   genre_id       10815 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 422.5+ KB


In [65]:
year_replace = {
    'year_released':{None:0}
}

In [66]:
game_title_df = game_title_df.replace(year_replace)
game_title_df

,game_title,year_released,publisher_id,genre_id
title_id,,,,
0,Wii Sports,2006.0,0,0
1,Mario Kart Wii,2008.0,0,1
2,Wii Sports Resort,2009.0,0,0
3,New Super Mario Bros.,2006.0,0,2
4,Wii Play,2006.0,0,3
...,...,...,...,...
10810,Chou Ezaru wa Akai Hana: Koi wa Tsuki ni Shiru...,2016.0,441,5
10811,Eiyuu Densetsu: Sora no Kiseki Material Collec...,2007.0,216,6
10812,Plushees,2008.0,136,4


In [67]:
game_title_df = game_title_df.astype({'year_released':int})

In [68]:
game_title_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10815 entries, 0 to 10814
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   game_title     10815 non-null  object
 1   year_released  10815 non-null  int32 
 2   publisher_id   10815 non-null  int64 
 3   genre_id       10815 non-null  int64 
dtypes: int32(1), int64(2), object(1)
memory usage: 380.2+ KB


#### Sales Table

In [36]:
#import transformed sales table by tyler in for platform to platform_id change
sales_df = pd.read_csv('sales.csv')
sales_df

,title_id,platform_id,na_sales,eu_sales,jp_sales,other_sales
0,0,Wii,41.49,29.02,3.77,8.46
1,1,Wii,15.85,12.88,3.79,3.31
2,2,Wii,15.75,11.01,3.28,2.96
3,3,DS,11.38,9.23,6.50,2.90
4,4,Wii,14.03,9.20,2.93,2.85
...,...,...,...,...,...,...
15794,10813,GBA,0.01,0.00,0.00,0.00
15795,2637,GC,0.01,0.00,0.00,0.00
15796,6110,PS2,0.00,0.00,0.00,0.00
15797,10814,DS,0.00,0.01,0.00,0.00


In [37]:
#reference previous code from testing section earlier
plat_replacer = {
    "platform_id": platformdict_s
}

sales_df = sales_df.replace(plat_replacer)
sales_df

,title_id,platform_id,na_sales,eu_sales,jp_sales,other_sales
0,0,0,41.49,29.02,3.77,8.46
1,1,0,15.85,12.88,3.79,3.31
2,2,0,15.75,11.01,3.28,2.96
3,3,1,11.38,9.23,6.50,2.90
4,4,0,14.03,9.20,2.93,2.85
...,...,...,...,...,...,...
15794,10813,5,0.01,0.00,0.00,0.00
15795,2637,14,0.01,0.00,0.00,0.00
15796,6110,4,0.00,0.00,0.00,0.00
15797,10814,1,0.00,0.01,0.00,0.00


In [38]:
sales_df = sales_df.set_index(['title_id','platform_id'])
sales_df

,,na_sales,eu_sales,jp_sales,other_sales
title_id,platform_id,,,,
0,0,41.49,29.02,3.77,8.46
1,0,15.85,12.88,3.79,3.31
2,0,15.75,11.01,3.28,2.96
3,1,11.38,9.23,6.50,2.90
4,0,14.03,9.20,2.93,2.85
...,...,...,...,...,...
10813,5,0.01,0.00,0.00,0.00
2637,14,0.01,0.00,0.00,0.00
6110,4,0.00,0.00,0.00,0.00


### Juncture Table

In [14]:
#Create juncture table of title_id's and platform_id's as some titles can be on multiple platforms
#using game title table, create dictionary of game titles and id's for replacement in original vgdf with
#subset of columns game_title and platform

game_dict = game_title_df[['game_title']].to_dict('index')

for key in game_dict:
    game_dict[key] = game_dict[key]["game_title"]
    
game_dict_s = {value:key for key, value in game_dict.items()}
game_dict_s 


{'Wii Sports': 0,
 'Mario Kart Wii': 1,
 'Wii Sports Resort': 2,
 'New Super Mario Bros.': 3,
 'Wii Play': 4,
 'New Super Mario Bros. Wii': 5,
 'Nintendogs': 6,
 'Mario Kart DS': 7,
 'Wii Fit': 8,
 'Wii Fit Plus': 9,
 'Kinect Adventures!': 10,
 'Grand Theft Auto V': 11,
 'Grand Theft Auto: San Andreas': 12,
 'Brain Age: Train Your Brain in Minutes a Day': 13,
 'Pokemon Diamond/Pokemon Pearl': 14,
 'Grand Theft Auto: Vice City': 15,
 'Pokemon Ruby/Pokemon Sapphire': 16,
 'Pokemon Black/Pokemon White': 17,
 'Brain Age 2: More Training in Minutes a Day': 18,
 'Gran Turismo 3: A-Spec': 19,
 'Call of Duty: Modern Warfare 3': 20,
 'Call of Duty: Black Ops': 21,
 'Pokemon X/Pokemon Y': 22,
 'Call of Duty: Black Ops 3': 23,
 'Call of Duty: Black Ops II': 24,
 'Call of Duty: Modern Warfare 2': 25,
 'Grand Theft Auto III': 26,
 'Super Smash Bros. Brawl': 27,
 'Animal Crossing: Wild World': 28,
 'Mario Kart 7': 29,
 'Halo 3': 30,
 'Pokemon HeartGold/Pokemon SoulSilver': 31,
 'Super Mario 64': 32,

In [15]:
#Create new DF with just title and platform from original DF with older platforms removed

title_plat_df = vgdf_plat_clean[['Name','Platform']]
title_plat_df

,Name,Platform
0,Wii Sports,Wii
2,Mario Kart Wii,Wii
3,Wii Sports Resort,Wii
6,New Super Mario Bros.,DS
7,Wii Play,Wii
...,...,...
16593,Woody Woodpecker in Crazy Castle 5,GBA
16594,Men in Black II: Alien Escape,GC
16595,SCORE International Baja 1000: The Official Game,PS2
16596,Know How 2,DS


In [16]:
#reference previous code from testing section earlier for changing both sets of game/plat names to id's for juncture table
title_plat_replacer = {
    "Name": game_dict_s,
    "Platform": platformdict_s
}

title_plat_id = title_plat_df.replace(title_plat_replacer)
title_plat_id = title_plat_id.rename(columns={'Name':'title_id','Platform':'platform_id'})
title_plat_id

,title_id,platform_id
0,0,0
2,1,0
3,2,0
6,3,1
7,4,0
...,...,...
16593,10813,5
16594,2637,14
16595,6110,4
16596,10814,1
